In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime

# Подготовка
Создадим словарь с User-Agent, чтобы авито не ловил нас сразу же.

In [2]:
base_url = 'https://www.avito.ru' 
url = 'https://www.avito.ru/sankt-peterburg/bytovaya_tehnika/dlya_kuhni/mikrovolnovye_pechi-ASgBAgICAkRglk_MB5pP?cd=1&pmax=4000&s=104&p={}'
ua = UserAgent()
headers = {'User-Agent': str(ua.chrome)}

# Вытащим ссылки на товары
Для этого я взял div с товаром и нашел в нем первую ссылку. Если пытаться просто вытаскивать ссылки с определенным классом, в них попадают также и ссылки на магазины.

In [3]:
r = requests.get(url.format(1), headers=headers)
soup = BeautifulSoup(r.text, features='lxml')
divs = soup.find_all('div', {'class':'item__line'})
links = [div.find_next('a') for div in divs]

In [4]:
links[0].get('href')

'/sankt-peterburg/bytovaya_tehnika/dostavka_svch_pechey_s_garantiey_lg_658_sanyo_77_b_1313002201'

Теперь, когда все работает, соберем это в функцию

In [5]:
def get_links(page):
    r = requests.get(url.format(page), headers=headers)
    if not r.ok:
        return []
    soup = BeautifulSoup(r.text, features='lxml')
    divs = soup.find_all('div', {'class':'item__line'})
    links = []
    if divs:
        links = [div.find_next('a') for div in divs]
        if links:
            links = [base_url + link.get('href') for link in links]
    return links

In [6]:
len(get_links(1))

52

# Получим номер последней страницы
Чтобы не вводить номер последней страницы вручную, будем получать его автоматически

In [7]:
def get_last_page(soup):
    next_button = soup.find('span', {'data-marker':'pagination-button/next'})
    last_page = next_button.find_previous_sibling('span')
    return int(last_page.text)

In [8]:
get_last_page(soup)

26

# Парсинг страницы
Для примера возьмем какую-нибудь микроволновку и начнем с ней работать

In [9]:
page_url = 'https://www.avito.ru/sankt-peterburg/bytovaya_tehnika/mikrovolnovaya_pech_samsung_1873727094'

r = requests.get(page_url, headers=headers)
soup = BeautifulSoup(r.text, features='lxml')

## Заголовок

In [10]:
title = soup.find('span', {'class':'title-info-title-text'}).text
title

'Микроволновая печь SAMSUNG'

## Цена

In [11]:
price = int(soup.find('span', {'class':'js-item-price'}).get('content'))
price

3000

## Имя продавца

In [12]:
seller_name = soup.find('div', {'class':'seller-info-name js-seller-info-name'}).text.strip()
seller_name

'Галина Румянцева'

# Рейтинг продавца
Поскольку рейтинг - вещественное число, если его нет, удобно воткнуть вместо него ``np.nan``

In [13]:
seller_rating = soup.find('span', {'class':'seller-info-rating-score'})
seller_rating = seller_rating.text.strip().replace(',', '.') if seller_rating else np.nan
seller_rating = float(seller_rating)
seller_rating

nan

## Количество отзывов

In [14]:
seller_review = soup.find('span', {'class':'seller-info-rating-caption'})
seller_review = seller_review.text.strip() if seller_review else '0'
seller_review = int(seller_review.split()[0])
seller_review

0

# Частное лицо / Компания

In [15]:
seller_status = soup.find_all('div', {'class':'seller-info-value'})[1]
seller_status = seller_status.find_previous_sibling('div').text.strip()
seller_status

'Частное лицо'

# Количество просмотров

In [16]:
views = soup.find('div', {'class':'title-info-metadata-item title-info-metadata-views'}).text
views = int(views[:views.find('(')].strip())
views

9

# Адрес

In [17]:
address = soup.find('span', {'class':'item-address__string'}).text.strip()
address

'Санкт-Петербург, Бухарестская ул.'

# Ближайшее метро

In [18]:
metro = soup.find('span', {'class':'item-address-georeferences-item__content'}).text.strip()
to_metro = soup.find('span', {'class':'item-address-georeferences-item__after'}).text.strip()
to_metro = ' '.join(to_metro.split())
print(metro, to_metro)

Проспект Славы 700 м


# Описание товара

In [19]:
desc = soup.find('div', {'class':'item-description-text'}).text.strip()
desc

'Микроволновая печь в отличном рабочем состоянии. Гриль+ Конвекция. Использовалась мало.'

# Время добавления

In [20]:
added_time = soup.find('div', {'class':'title-info-metadata-item-redesign'}).text.strip()
added_time

'25 марта в 16:20'

Теперь опять соберем все в одну функцию

In [21]:
def get_info(url):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, features='lxml')
    info = {}
    
    title = soup.find('span', {'class':'title-info-title-text'})
    if title:
        title = title.text
        info['title'] = title

    price = soup.find('span', {'class':'js-item-price'})
    if price:
        price = int(price.get('content'))
        info['price'] = price

    seller_name = soup.find('div', {'class':'seller-info-name js-seller-info-name'})
    if seller_name:
        seller_name = seller_name.text.strip()
        info['seller_name'] = seller_name
    
    seller_rating = soup.find('span', {'class':'seller-info-rating-score'})
    seller_rating = seller_rating.text.strip().replace(',', '.') if seller_rating else np.nan
    seller_rating = float(seller_rating)
    info['seller_rating'] = seller_rating
    
    seller_review = soup.find('span', {'class':'seller-info-rating-caption'})
    seller_review = seller_review.text.strip() if seller_review else '0'
    seller_review = int(seller_review.split()[0])
    info['seller_review'] = seller_review
    
    seller_status = soup.find_all('div', {'class':'seller-info-value'})
    if seller_status:
        seller_status = seller_status[1]
        seller_status = seller_status.find_previous_sibling('div').text.strip()
        info['seller_status'] = seller_status

    views = soup.find('div', {'class':'title-info-metadata-item title-info-metadata-views'})
    if views:
        views = views.text
        views = int(views[:views.find('(')].strip())
        info['views'] = views

    address = soup.find('span', {'class':'item-address__string'})
    if address:
        address = address.text.strip()
        info['address'] = address

    metro = soup.find('span', {'class':'item-address-georeferences-item__content'})
    metro = metro.text.strip() if metro else ''
    info['metro'] = metro
    
    to_metro = soup.find('span', {'class':'item-address-georeferences-item__after'})
    to_metro = to_metro.text.strip() if to_metro else ''
    to_metro = ' '.join(to_metro.split()) if to_metro else ''
    info['dist_to_metro'] = to_metro

    desc = soup.find('div', {'class':'item-description'})
    if desc:
        desc = desc.text.strip()
        info['description'] = desc

    added_time = soup.find('div', {'class':'title-info-metadata-item-redesign'})
    if added_time:
        added_time = added_time.text.strip()
        info['added_time'] = added_time
    
    info['link'] = url
    info['parsed_at'] = datetime.now()

    return info

In [22]:
get_info(page_url)

{'title': 'Микроволновая печь SAMSUNG',
 'price': 3000,
 'seller_name': 'Галина Румянцева',
 'seller_rating': nan,
 'seller_review': 0,
 'seller_status': 'Частное лицо',
 'views': 9,
 'address': 'Санкт-Петербург, Бухарестская ул.',
 'metro': 'Проспект Славы',
 'dist_to_metro': '700 м',
 'description': 'Микроволновая печь в отличном рабочем состоянии. Гриль+ Конвекция. Использовалась мало.',
 'added_time': '25 марта в 16:20',
 'link': 'https://www.avito.ru/sankt-peterburg/bytovaya_tehnika/mikrovolnovaya_pech_samsung_1873727094',
 'parsed_at': datetime.datetime(2020, 3, 31, 1, 9, 43, 964637)}